In [24]:
import copy
import pandas as pd
from elasticsearch import Elasticsearch

ES_HOST = "https://cluster.elasticsearch.dataesr.ovh/"
ES_INDEX = "bso-publications-20230403"
ES_PASSWORD = "vn84q9Xef9U7pmU"
ES_USER = "BSO"

es = Elasticsearch(ES_HOST, http_auth=(ES_USER, ES_PASSWORD))

### All publications in ES
With foreign publications, publications from HAL ...

In [25]:
count_es = es.count(index=ES_INDEX).get("count")
count_es

2961498

### Publications in the Monitor
Publications from France, with a DOI from crossref and of type "journal-article", "proceedings", "book-chapter", "book" or "preprint"

### Publications from the Monitor and analyzed by Datastet

In [72]:
body = {
	"query": {
		"bool": {
			"must": [
				{
					"term": {
						"bso_country.keyword": "fr"
					}
				},
				{
					"terms": {
						"id_type.keyword": [
							"doi"
						]
					}
				},
				{
					"terms": {
						"genre.keyword": [
							"journal-article",
							"proceedings",
							"book-chapter",
							"book",
							"preprint"
						]
					}
				},
                {"wildcard": {"oa_details.2022Q4.oa_host_type": "*"}}
			]
		}
	}
}
body['aggs'] = {'by_publication_year': {'terms': {'field': 'year', 'size':500}}}
body['size'] = 0
df = pd.DataFrame(es.search(index=ES_INDEX, body=body)['aggregations']['by_publication_year']['buckets'])
df.columns = ['year', 'bso']

In [73]:
body['query']['bool']['must'].append({'term': {'oa_details.2022Q4.is_oa': True}})
body['aggs'] = {'by_publication_year': {'terms': {'field': 'year', 'size':500}}}
body['size'] = 0
df2 = pd.DataFrame(es.search(index=ES_INDEX, body=body)['aggregations']['by_publication_year']['buckets'])
df2.columns = ['year', 'bso_oa']
df = pd.merge(df, df2, on='year')

In [74]:
body['query']['bool']['must'][-1] = {'term': {'oa_details.2022Q4.is_oa': False}}
body['aggs'] = {'by_publication_year': {'terms': {'field': 'year', 'size':500}}}
body['size'] = 0
df2 = pd.DataFrame(es.search(index=ES_INDEX, body=body)['aggregations']['by_publication_year']['buckets'])
df2.columns = ['year', 'bso_closed']
df = pd.merge(df, df2, on='year')
df

,year,bso,bso_oa,bso_closed
0,2020,175443,115394,60049
1,2019,168425,102038,66387
2,2018,166709,90810,75899
3,2017,160571,83461,77110
4,2021,160217,107722,52495
5,2016,157976,77823,80153
6,2022,154950,86375,68575
7,2015,146874,69104,77770
8,2014,146830,65552,81278
9,2013,143095,61849,81246


In [75]:
body['query']['bool']['must'][-1] = {'term': {'bso3_downloaded': 1}}
body['aggs'] = {'by_publication_year': {'terms': {'field': 'year', 'size':500}}}
body['size'] = 0
df2 = pd.DataFrame(es.search(index=ES_INDEX, body=body)['aggregations']['by_publication_year']['buckets'])
df2.columns = ['year', 'bso_download']
df = pd.merge(df, df2, on='year')
df

,year,bso,bso_oa,bso_closed,bso_download
0,2020,175443,115394,60049,119644
1,2019,168425,102038,66387,111702
2,2018,166709,90810,75899,109059
3,2017,160571,83461,77110,103248
4,2021,160217,107722,52495,103211
5,2016,157976,77823,80153,97913
6,2022,154950,86375,68575,50395
7,2015,146874,69104,77770,89668
8,2014,146830,65552,81278,88208
9,2013,143095,61849,81246,85914


In [76]:
body['query']['bool']['must'].append({'term': {'oa_details.2022Q4.is_oa': True}})
body['aggs'] = {'by_publication_year': {'terms': {'field': 'year', 'size':500}}}
body['size'] = 0
df2 = pd.DataFrame(es.search(index=ES_INDEX, body=body)['aggregations']['by_publication_year']['buckets'])
df2.columns = ['year', 'bso_oa_download']
df = pd.merge(df, df2, on='year')
df

,year,bso,bso_oa,bso_closed,bso_download,bso_oa_download
0,2020,175443,115394,60049,119644,100139
1,2019,168425,102038,66387,111702,89975
2,2018,166709,90810,75899,109059,78984
3,2017,160571,83461,77110,103248,72912
4,2021,160217,107722,52495,103211,85073
5,2016,157976,77823,80153,97913,67261
6,2022,154950,86375,68575,50395,34164
7,2015,146874,69104,77770,89668,58459
8,2014,146830,65552,81278,88208,55149
9,2013,143095,61849,81246,85914,52549


In [77]:
body['query']['bool']['must'][-1] = {'term': {'oa_details.2022Q4.is_oa': False}}
body['aggs'] = {'by_publication_year': {'terms': {'field': 'year', 'size':500}}}
body['size'] = 0
df2 = pd.DataFrame(es.search(index=ES_INDEX, body=body)['aggregations']['by_publication_year']['buckets'])
df2.columns = ['year', 'bso_closed_download']
df = pd.merge(df, df2, on='year')
df

,year,bso,bso_oa,bso_closed,bso_download,bso_oa_download,bso_closed_download
0,2020,175443,115394,60049,119644,100139,19505
1,2019,168425,102038,66387,111702,89975,21727
2,2018,166709,90810,75899,109059,78984,30075
3,2017,160571,83461,77110,103248,72912,30336
4,2021,160217,107722,52495,103211,85073,18138
5,2016,157976,77823,80153,97913,67261,30652
6,2022,154950,86375,68575,50395,34164,16231
7,2015,146874,69104,77770,89668,58459,31209
8,2014,146830,65552,81278,88208,55149,33059
9,2013,143095,61849,81246,85914,52549,33365


In [78]:
body['query']['bool']['must'][-1] = {'term': {'bso3_analyzed_grobid': 1}}
body['aggs'] = {'by_publication_year': {'terms': {'field': 'year', 'size':500}}}
body['size'] = 0
df2 = pd.DataFrame(es.search(index=ES_INDEX, body=body)['aggregations']['by_publication_year']['buckets'])
df2.columns = ['year', 'grobid']
df = pd.merge(df, df2, on='year')
df

,year,bso,bso_oa,bso_closed,bso_download,bso_oa_download,bso_closed_download,grobid
0,2020,175443,115394,60049,119644,100139,19505,105467
1,2019,168425,102038,66387,111702,89975,21727,98549
2,2018,166709,90810,75899,109059,78984,30075,86913
3,2017,160571,83461,77110,103248,72912,30336,83321
4,2021,160217,107722,52495,103211,85073,18138,83602
5,2016,157976,77823,80153,97913,67261,30652,79728
6,2022,154950,86375,68575,50395,34164,16231,34897
7,2015,146874,69104,77770,89668,58459,31209,72615
8,2014,146830,65552,81278,88208,55149,33059,66985
9,2013,143095,61849,81246,85914,52549,33365,66520


In [79]:
body['query']['bool']['must'][-1] = {'term': {'bso3_analyzed_softcite': 1}}
body['aggs'] = {'by_publication_year': {'terms': {'field': 'year', 'size':500}}}
body['size'] = 0
df2 = pd.DataFrame(es.search(index=ES_INDEX, body=body)['aggregations']['by_publication_year']['buckets'])
df2.columns = ['year', 'softcite']
df = pd.merge(df, df2, on='year')
df

,year,bso,bso_oa,bso_closed,bso_download,bso_oa_download,bso_closed_download,grobid,softcite
0,2020,175443,115394,60049,119644,100139,19505,105467,105270
1,2019,168425,102038,66387,111702,89975,21727,98549,98314
2,2018,166709,90810,75899,109059,78984,30075,86913,86725
3,2017,160571,83461,77110,103248,72912,30336,83321,83228
4,2021,160217,107722,52495,103211,85073,18138,83602,83456
5,2016,157976,77823,80153,97913,67261,30652,79728,79570
6,2022,154950,86375,68575,50395,34164,16231,34897,34869
7,2015,146874,69104,77770,89668,58459,31209,72615,72511
8,2014,146830,65552,81278,88208,55149,33059,66985,66835
9,2013,143095,61849,81246,85914,52549,33365,66520,66380


In [80]:
body['query']['bool']['must'][-1] = {'term': {'bso3_analyzed_datastet': 1}}
body['aggs'] = {'by_publication_year': {'terms': {'field': 'year', 'size':500}}}
body['size'] = 0
df2 = pd.DataFrame(es.search(index=ES_INDEX, body=body)['aggregations']['by_publication_year']['buckets'])
df2.columns = ['year', 'datastet']
df = pd.merge(df, df2, on='year')
df = df.sort_values(by='year')
df = df[df.year<2022]


In [82]:
for f in list(df.columns)[1:]:
    df.at[10, f] = df[f].sum()
df.at[10, 'year'] = 'Total'
df

,year,bso,bso_oa,bso_closed,bso_download,bso_oa_download,bso_closed_download,grobid,softcite,datastet
9,2013.0,143095.0,61849.0,81246.0,85914.0,52549.0,33365.0,66520.0,66380.0,57546.0
8,2014.0,146830.0,65552.0,81278.0,88208.0,55149.0,33059.0,66985.0,66835.0,58492.0
7,2015.0,146874.0,69104.0,77770.0,89668.0,58459.0,31209.0,72615.0,72511.0,61561.0
5,2016.0,157976.0,77823.0,80153.0,97913.0,67261.0,30652.0,79728.0,79570.0,67285.0
3,2017.0,160571.0,83461.0,77110.0,103248.0,72912.0,30336.0,83321.0,83228.0,70993.0
2,2018.0,166709.0,90810.0,75899.0,109059.0,78984.0,30075.0,86913.0,86725.0,74748.0
1,2019.0,168425.0,102038.0,66387.0,111702.0,89975.0,21727.0,98549.0,98314.0,77042.0
0,2020.0,175443.0,115394.0,60049.0,119644.0,100139.0,19505.0,105467.0,105270.0,82610.0
4,2021.0,160217.0,107722.0,52495.0,103211.0,85073.0,18138.0,83602.0,83456.0,71029.0
10,Total,1426140.0,773753.0,652387.0,908567.0,660501.0,248066.0,743700.0,742289.0,621306.0


In [83]:
for f in ['bso_download', 'grobid', 'softcite', 'datastet']:
    df[f'perc_{f}'] = df[f]/df['bso']
df['perc_bso_oa_download'] = df['bso_oa_download']/df['bso_oa']
df['perc_bso_closed_download'] = df['bso_closed_download']/df['bso_closed']
df

,year,bso,bso_oa,bso_closed,bso_download,bso_oa_download,bso_closed_download,grobid,softcite,datastet,perc_bso_download,perc_grobid,perc_softcite,perc_datastet,perc_bso_oa_download,perc_bso_closed_download
9,2013.0,143095.0,61849.0,81246.0,85914.0,52549.0,33365.0,66520.0,66380.0,57546.0,0.600398,0.464866,0.463888,0.402152,0.849634,0.410666
8,2014.0,146830.0,65552.0,81278.0,88208.0,55149.0,33059.0,66985.0,66835.0,58492.0,0.600749,0.456208,0.455186,0.398365,0.841302,0.406740
7,2015.0,146874.0,69104.0,77770.0,89668.0,58459.0,31209.0,72615.0,72511.0,61561.0,0.610510,0.494403,0.493695,0.419142,0.845957,0.401299
5,2016.0,157976.0,77823.0,80153.0,97913.0,67261.0,30652.0,79728.0,79570.0,67285.0,0.619797,0.504684,0.503684,0.425919,0.864282,0.382419
3,2017.0,160571.0,83461.0,77110.0,103248.0,72912.0,30336.0,83321.0,83228.0,70993.0,0.643005,0.518904,0.518325,0.442128,0.873606,0.393412
2,2018.0,166709.0,90810.0,75899.0,109059.0,78984.0,30075.0,86913.0,86725.0,74748.0,0.654188,0.521346,0.520218,0.448374,0.869772,0.396250
1,2019.0,168425.0,102038.0,66387.0,111702.0,89975.0,21727.0,98549.0,98314.0,77042.0,0.663215,0.585121,0.583726,0.457426,0.881779,0.327278
0,2020.0,175443.0,115394.0,60049.0,119644.0,100139.0,19505.0,105467.0,105270.0,82610.0,0.681954,0.601147,0.600024,0.470865,0.867801,0.324818
4,2021.0,160217.0,107722.0,52495.0,103211.0,85073.0,18138.0,83602.0,83456.0,71029.0,0.644195,0.521805,0.520894,0.443330,0.789746,0.345519
10,Total,1426140.0,773753.0,652387.0,908567.0,660501.0,248066.0,743700.0,742289.0,621306.0,0.637081,0.521478,0.520488,0.435656,0.853633,0.380244


In [15]:
df.to_csv('results.csv', index=False)
df

,year,bso,bso_oa,bso_closed,bso_download,bso_oa_download,bso_closed_download,grobid,softcite,datastet,perc_bso_download,perc_grobid,perc_softcite,perc_datastet,perc_bso_oa_download,perc_bso_closed_download
0,2013.0,143095.0,61849.0,81246.0,85914.0,52549.0,33365.0,66520.0,66380.0,57546.0,0.600398,0.464866,0.463888,0.402152,0.849634,0.410666
1,2014.0,146830.0,65552.0,81278.0,88208.0,55149.0,33059.0,66985.0,66835.0,58492.0,0.600749,0.456208,0.455186,0.398365,0.841302,0.406740
2,2015.0,146874.0,69104.0,77770.0,89668.0,58459.0,31209.0,72615.0,72511.0,61561.0,0.610510,0.494403,0.493695,0.419142,0.845957,0.401299
3,2016.0,157976.0,77823.0,80153.0,97913.0,67261.0,30652.0,79728.0,79570.0,67285.0,0.619797,0.504684,0.503684,0.425919,0.864282,0.382419
4,2017.0,160571.0,83461.0,77110.0,103248.0,72912.0,30336.0,83321.0,83228.0,70993.0,0.643005,0.518904,0.518325,0.442128,0.873606,0.393412
5,2018.0,166709.0,90810.0,75899.0,109059.0,78984.0,30075.0,86913.0,86725.0,74748.0,0.654188,0.521346,0.520218,0.448374,0.869772,0.396250
6,2019.0,168425.0,102038.0,66387.0,111702.0,89975.0,21727.0,98549.0,98314.0,77042.0,0.663215,0.585121,0.583726,0.457426,0.881779,0.327278
7,2020.0,175443.0,115394.0,60049.0,119644.0,100139.0,19505.0,105467.0,105270.0,82610.0,0.681954,0.601147,0.600024,0.470865,0.867801,0.324818
8,2021.0,160217.0,107722.0,52495.0,103211.0,85073.0,18138.0,83602.0,83456.0,71029.0,0.644195,0.521805,0.520894,0.443330,0.789746,0.345519
9,Total,1426140.0,773753.0,652387.0,908567.0,660501.0,248066.0,743700.0,742289.0,621306.0,0.637081,0.521478,0.520488,0.435656,0.853633,0.380244


In [34]:
import pandas as pd
import math
df = pd.read_csv('results.csv')
df.columns = ['Publication year', 'Number of publications', 'Number of OA publications', 'Number of closed access publications',
             'Number of publications downloaded', 
             'Number of OA publications downloaded',
             'Number of closed access publications downloaded', 
            'Number of publications analyzed with GROBID',
              'Number of publications analyzed with Softcite',
              'Number of publications analyzed with DataStet',
             '% of publications downloaded', 
              '% of publications analyzed with GROBID', 
               '% of publications analyzed with Softcite',
               '% of publications analyzed with DataStet', 
              '% of OA publications downloaded', '% of closed access publications downloaded']

for k in list(df.columns):
    if '%' in k:
        df[k] = round(100 * df[k], 1)
    elif 'Number' in k:
        df[k] = df[k].astype(int)

#df['bso'] - df['bso_oa'] - df['bso_closed']
df1 = df[['Publication year', 'Number of publications', 'Number of OA publications', 
    'Number of closed access publications'
   ]]
df2 = df[['Publication year',  '% of publications downloaded', 
    '% of OA publications downloaded', '% of closed access publications downloaded'
   ]]

In [36]:
df2.to_clipboard(index=False, sep='&')

In [37]:
df3 = df[['Publication year', 
          'Number of publications downloaded',
          'Number of publications analyzed with GROBID',
              'Number of publications analyzed with Softcite',
              'Number of publications analyzed with DataStet',
   ]]
df3.to_clipboard(index=False, sep='&')

### Publications from the Monitor and analyzed by Softcite

In [ ]:
body_copy = copy.deepcopy(body)
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "softcite_details" } })
count_bso_datastet = es.count(index=ES_INDEX, body=body_copy).get("count")
count_bso_datastet

### Publications from the Monitor and analyzed by DataStet AND Softcite

In [ ]:
body_copy = copy.deepcopy(body)
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "datastet_details" } })
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "softcite_details" } })
count_bso_datastet_softcite = es.count(index=ES_INDEX, body=body_copy).get("count")
count_bso_datastet_softcite

### Publications in ES and analyzed by DataStet

In [ ]:
body2 = {
	"query": {
		"bool": {
			"must": [
			]
		}
	}
}

body_copy = copy.deepcopy(body2)
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "datastet_details" } })
count_es_datastet = es.count(index=ES_INDEX, body=body_copy).get("count")
count_es_datastet

### Publications in ES and analyzed by Softcite

In [ ]:
body_copy = copy.deepcopy(body2)
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "softcite_details" } })
count_es_datastet = es.count(index=ES_INDEX, body=body_copy).get("count")
count_es_datastet

### Publications in ES and analyzed by DataStet AND Softcite

In [ ]:
body_copy = copy.deepcopy(body2)
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "datastet_details" } })
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "softcite_details" } })
count_es_datastet = es.count(index=ES_INDEX, body=body_copy).get("count")
count_es_datastet